# Moving and saving logins

Sometime, you would like to login in with one instance of `EmpowerHandler`, and use the login in another instance. Examples could be when the user logs into a frontend, and you want to interact with Empower in a backend, or if you want to save the logins in a database to save time logging in.

This notebook suggests how to work with that.

The minimum thing you need to do is to transfer the token and session ID. Note that it is a good idea to overwrite the token and session ID from the `EmpowerHandler` after you have extracted it - `EmpowerHandler` logs out when it is deleted, which happens automatically as part of Python garbage collection if it is no longer referenced anywhere.

In [ ]:
import os
from dotenv import load_dotenv
from OptiHPLCHandler import EmpowerHandler

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")


In [ ]:
handler = EmpowerHandler(
    project="WebAPI_test",
    address=EMPOWER_API_ADDRESS,
    allow_login_without_context_manager=True,
)
handler.login()
token = handler.connection.token
session_id = handler.connection.session_id
handler.connection.token = None
handler.connection.session_id = None

You can now create a new EmpowerHandler and pass the token and session ID into it.

Note that the new handler will lack some conviniences that are initialised during `login()`. Specifically, it will not have the Empower display names of SampleSetLineField set as synonyms, and it doesn't know about enumerated custom SampleSetLineFields. This means that, when creating SampleSetMethods with `PostExperiment`, you have to use the API names for SampleSetLineFields, and that you have to input the values of enumerated custom fields as `{"member": value}`.

You can get the synonyms and enumerated fields from the API by running the method `SetSynonymsAndEnumeratedFields()`, or you can set them manually if you know which synonyms and enumerated custom fields you are going to use.

You can manually add synonyms by appending to the synonym dict with `handler.synonym_dict[display_name] = API_name`.

You can tell the `EmpowerHandler` about individual enumrated fields by running `SetAllowedSamplesetLineFieldValues`. If you don't give `allowed_values`, this method will get them from the API. If `allowed_values` is an empty tuple, validation of this field is turned off. If it is a list of strings, posting to this field is only allowed if the posted value is in the list.

In [ ]:
new_handler = EmpowerHandler(
    project="WebAPI_test",
    address=EMPOWER_API_ADDRESS,
)
new_handler.connection.token = token
new_handler.connection.session_id = session_id
# Handler is now "logged in" and can be used to interact with Empower
new_handler.synonym_dict["Display Name In Empower"] = "display_name_in_empower"
# You need to know what the API name is for the field you want to use
new_handler.SetAllowedSamplesetLineFieldValues(
    field_name="Display Name In Empower", # Synonyms are allowed here
    allowed_values=["value1", "value2"], # This turns validation on for the field
)
new_handler.SetAllowedSamplesetLineFieldValues(
    field_name="Display Name In Empower",
    allowed_values=tuple(), # An empty tupple turns validation off for the field
)
new_handler.SetAllowedSamplesetLineFieldValues(field_name="Display Name In Empower")
# By not giving the allowed names, they are looked up in Empower.

Note that `EmpowerHandler` will refresh the token if it has expired when it tries to use it. So if you save the token for future use, you might want to extract it from the handler again when you are done with it - The token in the handler might not be the same as the one you put in, and in that case, the token you put in can no longer be used, and can't be renewed.